In [27]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.5 MB/s eta 0:00:00


In [5]:
import google.colab
google.colab.drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [7]:
data_path = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'

In [8]:
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

# train_ft = pd.read_csv(data_path + 'train_ft_97983.csv')
train_ft = pd.read_csv(data_path + 'train_120000_ver1.csv')

In [9]:
train_ft

,category,question,answer
0,건축구조,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,건축구조,내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다."
2,건축구조,철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...
3,건축구조,철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ..."
4,건축구조,철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.
...,...,...,...
119995,기타+시공,"새집증후군이 무엇이며, 외부에 소리가 안나가게 하는 목적으로 차음재만 설치해도 돼?","새집증후군이란 주택, 및 건축물 등 생활환경 속에서 일어나는 화학물질 과민증으로 신..."
119996,마감재+인테리어,"써모사이딩이 무엇이며, 인테리어 디자인에서 조명의 역할은 무엇인가요?",써모사이딩은 메탈사이딩이라고도 불리는 외장재이며 알루미늄 합금 강판에 고내후성 도료...
119997,마감하자,"훼손이 무엇이고, 공간 내 높은습도로 인해 도배지에 얼룩이 발생할 수 있어?","훼손은 터짐 하자, 몰딩수정 등 물리적인 충격이나 힘에 의해 발생하는 하자입니다. ..."
119998,기타+마감하자,"새집증후군의 원인이 무엇이고, 벽지에 반점이 생기는 원인이 뭐야?",새집증후군은 새솔 지은 집에 입주했을 때 이전에 없던 이상 증상이 신체에 나타나는 ...


In [10]:
train_ft['category'] = train_ft['category'].str.replace(' ','').str.split('+')

In [11]:
train_ft['category'].value_counts()

[마감재]            30081
[마감재, 인테리어]       8761
[마감재, 시공]         7839
[인테리어]            7772
[시공]              6829
[인테리어, 마감재]       6092
[시공, 마감재]         6070
[마감재, 마감하자]       4190
[마감하자]            2934
[시공, 인테리어]        2757
[인테리어, 시공]        2521
[마감하자, 마감재]       2520
[건축구조, 마감재]       2251
[마감재, 건축구조]       2200
[마감재, 기타]         2000
[기타, 마감재]         1857
[타마감하자, 마감재]      1683
[건축구조]            1469
[마감재, 타마감하자]      1385
[시공, 마감하자]        1360
[인테리어, 마감하자]      1301
[기타]              1267
[마감하자, 인테리어]      1160
[마감하자, 시공]        1013
[건축구조, 인테리어]      1003
[건축구조, 시공]         949
[타마감하자]            921
[기타, 인테리어]         865
[타마감하자, 인테리어]      777
[기타, 시공]           768
[인테리어, 건축구조]       754
[타마감하자, 시공]        683
[시공, 건축구조]         670
[인테리어, 기타]         656
[시공, 기타]           590
[건축구조, 마감하자]       499
[인테리어, 타마감하자]      472
[시공, 타마감하자]        466
[타마감하자, 마감하자]      398
[기타, 마감하자]         379
[마감하자, 건축구조]       292
[마감하자, 기타]         259
[건축구조, 기타]         234
[마감하자, 타마감하

In [12]:
for i in tqdm(range(len(train_ft['category']))):
    if len(train_ft['category'][i]) == 2 and train_ft['category'][i][0] == train_ft['category'][i][1]:
        train_ft['category'][i] = [train_ft['category'][i][0]]

  0%|          | 0/120000 [00:00<?, ?it/s]

# 1) 키워드(kiwi) 카테고리

In [13]:
train_ft.head()

,category,question,answer
0,[건축구조],면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,[건축구조],내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다."
2,[건축구조],철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...
3,[건축구조],철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ..."
4,[건축구조],철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.


In [14]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.17.0-py3-none-any.whl size=34845034 sha256=ab7f5bf3f1cfecfd04791c0ea8bc40a44e2b274b814bc05bcc26777692add8fa
  Stored in directory: /root/.cache/pip/wheels/90/bb/ae/15e1cf26d17183040bac268fbf8d1ca1f45f7f4daa0fb76b91
Successfully built kiwipiepy-model


In [15]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 37.5 MB/s eta 0:00:00


# 형태소 분석 키워드 추출

In [16]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
kiwi.tokenize(train_ft['question'][0])

[Token(form='면', tag='NNG', start=0, len=1),
 Token(form='지', tag='VV', start=1, len=1),
 Token(form='ᆫ', tag='ETM', start=1, len=1),
 Token(form='장치', tag='NNG', start=2, len=2),
 Token(form='가', tag='JKS', start=4, len=1),
 Token(form='뭐', tag='NP', start=6, len=1),
 Token(form='이', tag='VCP', start=7, len=0),
 Token(form='야', tag='EF', start=7, len=1),
 Token(form='?', tag='SF', start=8, len=1)]

In [31]:
from konlpy.tag import Hannanum

hannanum = Hannanum()
hannanum.pos(train_ft['question'][0])

[('면진장치', 'N'), ('가', 'J'), ('뭐', 'N'), ('이', 'J'), ('야', 'E'), ('?', 'S')]

In [20]:
text = train_ft['question'][1]

print(text)
print(kiwi.tokenize(text))
print(okt.pos(text))
print(hannanum.pos(text)) # 채택!
print(komoran.pos(text))

내진설계의 종류 좀 알려줘
[Token(form='내진', tag='NNG', start=0, len=2), Token(form='설계', tag='NNG', start=2, len=2), Token(form='의', tag='JKG', start=4, len=1), Token(form='종류', tag='NNG', start=6, len=2), Token(form='좀', tag='MAG', start=9, len=1), Token(form='알려주', tag='VV', start=11, len=3), Token(form='어', tag='EF', start=13, len=1)]
[('내', 'Determiner'), ('진설', 'Noun'), ('계', 'Suffix'), ('의', 'Josa'), ('종류', 'Noun'), ('좀', 'Noun'), ('알려줘', 'Verb')]
[('내진설계', 'N'), ('의', 'J'), ('종류', 'N'), ('좀', 'M'), ('알', 'P'), ('려', 'E'), ('주', 'P'), ('어', 'E')]
[('내진', 'NNP'), ('설계', 'NNP'), ('의', 'JKG'), ('종류', 'NNG'), ('좀', 'MAG'), ('알리', 'VV'), ('어', 'EC'), ('주', 'VX'), ('어', 'EC')]


In [21]:
for i in range(100) :
    print(train_ft['question'][i])
    pos_result = hannanum.pos(train_ft['question'][i])
    # print(hannanum.pos(train_ft['question'][i]))
    nouns = [word for word, pos in pos_result if pos.startswith('N')]
    print(nouns)
    print()


면진장치가 뭐야?
['면진장치', '뭐']

내진설계의 종류 좀 알려줘
['내진설계', '종류']

철골구조의 장점이 뭐야?
['철골구조', '장점', '뭐']

철골철근 콘크리트 구조가 뭐야?
['철골철근', '콘크리트', '구조', '뭐']

철골구조는 어떤 방식이 있어?
['철골구조', '방식']

커튼월이 뭐야?
['커튼월', '뭐']

내진구조가 뭐야?
['내진구조', '뭐']

중목구조 방식이 뭐야?
['중목구조', '방식', '뭐']

기둥-보 구조 방식이 뭐야?
['기둥', '보', '구조', '방식', '뭐']

블록구조가 뭐야
['블록구조', '뭐']

철골구조의 단점이 뭐야?
['철골구조', '단점', '뭐']

콘크리트 구조는 어떤 방식이 있어?
['콘크리트', '구조', '방식']

철골콘크리트 구조가 뭐야?
['철골콘크리트', '구조', '뭐']

면진구조가 뭐야?
['면진구조', '뭐']

철골구조가 뭐야?
['철골구조', '뭐']

면진구조와 제진구조의 차이점이 뭐야?
['면진구조', '진구조의', '차이점', '뭐']

철근콘크리트 구조가 뭐야?
['철근콘크리트', '구조', '뭐']

건축물의 구조는 어떻게 구분해?
['건축물', '구조', '구분']

조적식 구조는 어떤 방식이 있어?
['조적식', '구조', '방식']

목구조는 어떤 방식이 있어?
['목구조', '방식']

경량철골구조가 뭐야?
['경량철골구조', '뭐']

석구조가 뭐야?
['석구조', '뭐']

화재에 취약한 철골구조의 보완법에 뭐가 있을까?
['화재', '취약', '철골구조', '보완법', '뭐']

프리케스트콘크리트 구조가 뭐야?
['프리케스트콘크리트', '구조', '뭐']

제진구조가 뭐야?
['진구조', '뭐']

통나무구조 방식이 뭐야?
['통나무구조', '방식', '뭐']

일반철골구조가 뭐야?
['일반철골구조', '뭐']

내진설계에서 안정성이 높은 순서대로 말해줘
['내진설계', '안정성', '순서', '말']

경량목구조 방식이 뭐야?
[

In [53]:
hann_answers = []

for i in tqdm(range(train_ft.shape[0])) :
    pos_result = hannanum.pos(train_ft['answer'][i])
    nouns = [word for word, pos in pos_result if pos.startswith('N')]
    hann_answers.append(nouns)

train_ft['hann_answers'] = hann_answers


100%|██████████| 120000/120000 [29:43<00:00, 67.27it/s]


In [35]:
hann_questions = []

for i in tqdm(range(train_ft.shape[0])) :
    # print(train_ft['question'][i])
    pos_result = hannanum.pos(train_ft['question'][i])
    # print(hannanum.pos(train_ft['question'][i]))
    nouns = [word for word, pos in pos_result if pos.startswith('N')]
    hann_questions.append(nouns)


100%|██████████| 120000/120000 [05:48<00:00, 344.27it/s]


In [58]:

kiwi_answers = []

for answer in tqdm(train_ft['answer']):
    results = []  # 결과 리스트를 각 리뷰마다 초기화
    result = kiwi.analyze(answer)
    for token, pos, _, _ in result[0][0]:
        if pos in ['NNG']:  # 일반명사, 형용사, 동사, 어근
            results.append(token)
    kiwi_answers.append(results)

train_ft['kiwi_answers'] = kiwi_answers

100%|██████████| 120000/120000 [18:01<00:00, 110.94it/s]


In [ ]:

kiwi_questions = []

for question in tqdm(train_ft['question']):
    results = []  # 결과 리스트를 각 리뷰마다 초기화
    result = kiwi.analyze(question)
    for token, pos, _, _ in result[0][0]:
        if pos in ['NNG']:  # 일반명사, 형용사, 동사, 어근
            results.append(token)
    kiwi_questions.append(results)

In [60]:
train_ft

,category,question,answer,kiwi_questions,hann_questions,hann_answers,kiwi_answers
0,[건축구조],면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"[면, 장치]","[면진장치, 뭐]","[면진장치란, 지반, 진동, 에너지, 흡수, 건물, 진동, 진동, 격리장치]","[면, 장치, 지반, 진동, 에너지, 흡수, 건물, 진동, 진동, 격리, 장치]"
1,[건축구조],내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","[내진, 설계, 종류]","[내진설계, 종류]","[설계, 종류, 내진구조,, 진구조, 면진구조]","[내진, 설계, 종류, 내진, 구조, 진, 구조, 면, 구조]"
2,[건축구조],철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,"[철골, 구조, 장점]","[철골구조, 장점, 뭐]","[철골구조, 건물, 외벽, 하중, 때문, 고층, 건물, 것, 축, 가능, 것, 장점]","[철골, 구조, 건물, 외벽, 하중, 고층, 건물, 건축, 가능, 장점]"
3,[건축구조],철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","[철골, 철근, 콘크리트, 구조]","[철골철근, 콘크리트, 구조, 뭐]","[철근철골콘크리트, 철골, 철근, 콘크리트, 사용, 것, 축, 구조, 철골, 소재,...","[철근, 철골, 콘크리트, 철골, 철근, 콘크리트, 사용, 건축, 구조, 철골, 소..."
4,[건축구조],철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.,"[철골, 구조, 방식]","[철골구조, 방식]","[철골구조, 일반철골구조, 경량철골구조]","[철골, 구조, 일반, 철골, 구조, 경량, 철골, 구조]"
...,...,...,...,...,...,...,...
119995,"[기타, 시공]","새집증후군이 무엇이며, 외부에 소리가 안나가게 하는 목적으로 차음재만 설치해도 돼?","새집증후군이란 주택, 및 건축물 등 생활환경 속에서 일어나는 화학물질 과민증으로 신...","[새집, 증후군, 외부, 소리, 목적, 차음재, 설치]","[새집증후군, 무엇, 외부, 소리, 안나가, 목적, 차음재만, 설치해]","[새집증후군, 주택, 건축물, 등, 생활환경, 속, 화학물질, 과민증, 신축, 리모...","[새집, 증후군, 주택, 건축물, 생활환경, 속, 화학, 물질, 신축, 리모델링, ..."
119996,"[마감재, 인테리어]","써모사이딩이 무엇이며, 인테리어 디자인에서 조명의 역할은 무엇인가요?",써모사이딩은 메탈사이딩이라고도 불리는 외장재이며 알루미늄 합금 강판에 고내후성 도료...,"[인테리어, 디자인, 조명, 역할]","[써모사이딩, 무엇, 인테리, 디자인, 조명, 역할, 무엇]","[써모사이딩, 메탈사이딩, 외장재, 알루미늄, 합금, 강판, 고내후성, 도료, 코팅...","[써모사이딩은, 메탈, 사이, 외장재이며, 알루미늄, 합금, 강판, 고내후성, 도료..."
119997,[마감하자],"훼손이 무엇이고, 공간 내 높은습도로 인해 도배지에 얼룩이 발생할 수 있어?","훼손은 터짐 하자, 몰딩수정 등 물리적인 충격이나 힘에 의해 발생하는 하자입니다. ...","[훼손, 공간, 습도, 도배지, 얼룩, 발생]","[훼손, 무엇, 공간, 내, 높은습도, 도배지, 얼룩, 발생, 수]","[훼손, 몰딩수정, 등, 물리적, 충격, 힘, 발생, 하자, 습, 환경, 도배지, ...","[훼손, 몰딩, 수정, 물리, 충격, 힘, 발생, 하자, 습도, 환경, 도배지, 표..."
119998,"[기타, 마감하자]","새집증후군의 원인이 무엇이고, 벽지에 반점이 생기는 원인이 뭐야?",새집증후군은 새솔 지은 집에 입주했을 때 이전에 없던 이상 증상이 신체에 나타나는 ...,"[새집, 증후군, 원인, 벽지, 반점, 원인]","[새집증후군, 원인, 무엇, 벽지, 반점, 원인, 뭐]","[새집증후군, 새솔, 집, 입주, 때, 이전, 이상, 증상, 신체, 것, 통칭, 휘...","[새집, 증후군, 솔, 집, 입주, 때, 이전, 이상, 증상, 신체, 통칭, 휘발,..."


In [29]:
train_ft['kiwi_questions'] = kiwi_questions

In [37]:
train_ft['hann_questions'] = hann_questions
train_ft

,category,question,answer,kiwi_questions,hann_questions
0,[건축구조],면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"[면, 장치]","[면진장치, 뭐]"
1,[건축구조],내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","[내진, 설계, 종류]","[내진설계, 종류]"
2,[건축구조],철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,"[철골, 구조, 장점]","[철골구조, 장점, 뭐]"
3,[건축구조],철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","[철골, 철근, 콘크리트, 구조]","[철골철근, 콘크리트, 구조, 뭐]"
4,[건축구조],철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.,"[철골, 구조, 방식]","[철골구조, 방식]"
...,...,...,...,...,...
119995,"[기타, 시공]","새집증후군이 무엇이며, 외부에 소리가 안나가게 하는 목적으로 차음재만 설치해도 돼?","새집증후군이란 주택, 및 건축물 등 생활환경 속에서 일어나는 화학물질 과민증으로 신...","[새집, 증후군, 외부, 소리, 목적, 차음재, 설치]","[새집증후군, 무엇, 외부, 소리, 안나가, 목적, 차음재만, 설치해]"
119996,"[마감재, 인테리어]","써모사이딩이 무엇이며, 인테리어 디자인에서 조명의 역할은 무엇인가요?",써모사이딩은 메탈사이딩이라고도 불리는 외장재이며 알루미늄 합금 강판에 고내후성 도료...,"[인테리어, 디자인, 조명, 역할]","[써모사이딩, 무엇, 인테리, 디자인, 조명, 역할, 무엇]"
119997,[마감하자],"훼손이 무엇이고, 공간 내 높은습도로 인해 도배지에 얼룩이 발생할 수 있어?","훼손은 터짐 하자, 몰딩수정 등 물리적인 충격이나 힘에 의해 발생하는 하자입니다. ...","[훼손, 공간, 습도, 도배지, 얼룩, 발생]","[훼손, 무엇, 공간, 내, 높은습도, 도배지, 얼룩, 발생, 수]"
119998,"[기타, 마감하자]","새집증후군의 원인이 무엇이고, 벽지에 반점이 생기는 원인이 뭐야?",새집증후군은 새솔 지은 집에 입주했을 때 이전에 없던 이상 증상이 신체에 나타나는 ...,"[새집, 증후군, 원인, 벽지, 반점, 원인]","[새집증후군, 원인, 무엇, 벽지, 반점, 원인, 뭐]"


In [45]:
kiwi_question_vocabulary = []

for i in tqdm(kiwi_questions) :
    for j in i :
        kiwi_question_vocabulary.append(j)

hann_question_vocabulary = []

for i in tqdm(hann_questions) :
    for j in i :
        hann_question_vocabulary.append(j)


100%|██████████| 120000/120000 [00:00<00:00, 834291.66it/s]


In [50]:
from collections import Counter
kiwi_question_vocabulary_tuple = tuple(kiwi_question_vocabulary)

Counter(kiwi_question_vocabulary_tuple).most_common()

[('장점', 24771),
 ('단점', 21097),
 ('인테리어', 16630),
 ('벽지', 12517),
 ('페인트', 10941),
 ('구조', 10752),
 ('도배지', 10514),
 ('때', 10433),
 ('타일', 9152),
 ('공간', 8933),
 ('사용', 8882),
 ('발생', 8544),
 ('도배', 8234),
 ('방법', 7128),
 ('벽돌', 6674),
 ('결', 6618),
 ('종류', 6224),
 ('마루', 6183),
 ('주의', 6064),
 ('디자인', 5993),
 ('시공', 5974),
 ('설명', 5820),
 ('공사', 5728),
 ('단열', 5593),
 ('선택', 5413),
 ('점', 5330),
 ('콘크리트', 5109),
 ('원인', 4973),
 ('작업', 4827),
 ('얼마', 4635),
 ('사이', 4635),
 ('판', 4464),
 ('수명', 4446),
 ('활용', 4376),
 ('방수', 4275),
 ('효과', 4249),
 ('조명', 4029),
 ('기대', 3961),
 ('법', 3927),
 ('경우', 3767),
 ('바닥재', 3497),
 ('방식', 3466),
 ('세라믹', 3442),
 ('고려', 3436),
 ('스타일', 3419),
 ('발', 3367),
 ('대책', 3348),
 ('철골', 3241),
 ('보온', 3229),
 ('폼', 3167),
 ('벽', 3156),
 ('집', 3015),
 ('벽면', 3004),
 ('부위', 2986),
 ('곰팡이', 2894),
 ('추천', 2847),
 ('실내', 2822),
 ('설치', 2811),
 ('흡음재', 2801),
 ('장판', 2768),
 ('노출', 2690),
 ('점토', 2665),
 ('수제', 2542),
 ('재료', 2511),
 ('실크', 2502),
 ('면', 2476),


In [51]:
from collections import Counter
hann_question_vocabulary_tuple = tuple(hann_question_vocabulary)

Counter(hann_question_vocabulary_tuple).most_common()

[('뭐', 78851),
 ('무엇', 54559),
 ('장점', 24721),
 ('단점', 21037),
 ('수', 20833),
 ('인테리어', 10889),
 ('도배지', 10283),
 ('때', 9670),
 ('공간', 8933),
 ('사용', 8852),
 ('발생', 8544),
 ('중', 7421),
 ('결', 7169),
 ('방법', 6750),
 ('것', 6669),
 ('페인트', 6426),
 ('종류', 6214),
 ('디자인', 5993),
 ('설명', 5810),
 ('인테리', 5741),
 ('타일', 5631),
 ('벽지', 5420),
 ('선택', 5393),
 ('점', 5235),
 ('시공', 5100),
 ('원인', 4963),
 ('얼마', 4645),
 ('도배', 4605),
 ('활용', 4376),
 ('흡음재', 4009),
 ('기대수명', 3831),
 ('조명', 3785),
 ('경우', 3767),
 ('바닥재', 3497),
 ('구조', 3487),
 ('대책', 3348),
 ('보온판', 3229),
 ('스타일', 3170),
 ('주의해', 3031),
 ('벽면', 3004),
 ('부위', 2986),
 ('집', 2912),
 ('곰팡이', 2894),
 ('벽', 2890),
 ('추천', 2847),
 ('효과적', 2701),
 ('어디', 2651),
 ('방식', 2647),
 ('설치', 2557),
 ('장판', 2495),
 ('배관공사', 2461),
 ('다공질', 2431),
 ('샤시', 2408),
 ('노출콘크리트', 2393),
 ('주의', 2380),
 ('세라믹', 2335),
 ('내단열', 2271),
 ('실크벽지', 2254),
 ('실내', 2246),
 ('습도', 2158),
 ('외단열', 2156),
 ('천장', 2139),
 ('처리', 2132),
 ('비드법', 2123),
 ('발수제', 2123)

In [ ]:
class_folders = ['가구수정', '걸레받이수정', '곰팡이', '꼬임', '오염', '녹오염', '들뜸', '면불량', '몰딩수정', '반점', '석고수정', '오타공', '울음', '이음부불량', '창틀,문틀수정', '터짐', '틈새과다', '피스', '훼손']


In [61]:
train_ft.to_csv('train_120000_ver1_pos_analyze.csv', index=False, encoding='utf-8-sig' )

In [62]:
train_ft.to_csv(f'{data_path}train_120000_ver1_pos_analyze.csv', index=False, encoding='utf-8-sig' )

In [63]:
train_ft

,category,question,answer,kiwi_questions,hann_questions,hann_answers,kiwi_answers
0,[건축구조],면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"[면, 장치]","[면진장치, 뭐]","[면진장치란, 지반, 진동, 에너지, 흡수, 건물, 진동, 진동, 격리장치]","[면, 장치, 지반, 진동, 에너지, 흡수, 건물, 진동, 진동, 격리, 장치]"
1,[건축구조],내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","[내진, 설계, 종류]","[내진설계, 종류]","[설계, 종류, 내진구조,, 진구조, 면진구조]","[내진, 설계, 종류, 내진, 구조, 진, 구조, 면, 구조]"
2,[건축구조],철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,"[철골, 구조, 장점]","[철골구조, 장점, 뭐]","[철골구조, 건물, 외벽, 하중, 때문, 고층, 건물, 것, 축, 가능, 것, 장점]","[철골, 구조, 건물, 외벽, 하중, 고층, 건물, 건축, 가능, 장점]"
3,[건축구조],철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","[철골, 철근, 콘크리트, 구조]","[철골철근, 콘크리트, 구조, 뭐]","[철근철골콘크리트, 철골, 철근, 콘크리트, 사용, 것, 축, 구조, 철골, 소재,...","[철근, 철골, 콘크리트, 철골, 철근, 콘크리트, 사용, 건축, 구조, 철골, 소..."
4,[건축구조],철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.,"[철골, 구조, 방식]","[철골구조, 방식]","[철골구조, 일반철골구조, 경량철골구조]","[철골, 구조, 일반, 철골, 구조, 경량, 철골, 구조]"
...,...,...,...,...,...,...,...
119995,"[기타, 시공]","새집증후군이 무엇이며, 외부에 소리가 안나가게 하는 목적으로 차음재만 설치해도 돼?","새집증후군이란 주택, 및 건축물 등 생활환경 속에서 일어나는 화학물질 과민증으로 신...","[새집, 증후군, 외부, 소리, 목적, 차음재, 설치]","[새집증후군, 무엇, 외부, 소리, 안나가, 목적, 차음재만, 설치해]","[새집증후군, 주택, 건축물, 등, 생활환경, 속, 화학물질, 과민증, 신축, 리모...","[새집, 증후군, 주택, 건축물, 생활환경, 속, 화학, 물질, 신축, 리모델링, ..."
119996,"[마감재, 인테리어]","써모사이딩이 무엇이며, 인테리어 디자인에서 조명의 역할은 무엇인가요?",써모사이딩은 메탈사이딩이라고도 불리는 외장재이며 알루미늄 합금 강판에 고내후성 도료...,"[인테리어, 디자인, 조명, 역할]","[써모사이딩, 무엇, 인테리, 디자인, 조명, 역할, 무엇]","[써모사이딩, 메탈사이딩, 외장재, 알루미늄, 합금, 강판, 고내후성, 도료, 코팅...","[써모사이딩은, 메탈, 사이, 외장재이며, 알루미늄, 합금, 강판, 고내후성, 도료..."
119997,[마감하자],"훼손이 무엇이고, 공간 내 높은습도로 인해 도배지에 얼룩이 발생할 수 있어?","훼손은 터짐 하자, 몰딩수정 등 물리적인 충격이나 힘에 의해 발생하는 하자입니다. ...","[훼손, 공간, 습도, 도배지, 얼룩, 발생]","[훼손, 무엇, 공간, 내, 높은습도, 도배지, 얼룩, 발생, 수]","[훼손, 몰딩수정, 등, 물리적, 충격, 힘, 발생, 하자, 습, 환경, 도배지, ...","[훼손, 몰딩, 수정, 물리, 충격, 힘, 발생, 하자, 습도, 환경, 도배지, 표..."
119998,"[기타, 마감하자]","새집증후군의 원인이 무엇이고, 벽지에 반점이 생기는 원인이 뭐야?",새집증후군은 새솔 지은 집에 입주했을 때 이전에 없던 이상 증상이 신체에 나타나는 ...,"[새집, 증후군, 원인, 벽지, 반점, 원인]","[새집증후군, 원인, 무엇, 벽지, 반점, 원인, 뭐]","[새집증후군, 새솔, 집, 입주, 때, 이전, 이상, 증상, 신체, 것, 통칭, 휘...","[새집, 증후군, 솔, 집, 입주, 때, 이전, 이상, 증상, 신체, 통칭, 휘발,..."


In [65]:
train_ft['kiwi_questions_str'] = train_ft['kiwi_questions'].apply(lambda x: ', '.join(map(str, x)))
train_ft['kiwi_answers_str'] = train_ft['kiwi_answers'].apply(lambda x: ', '.join(map(str, x)))


In [67]:
train_ft.to_csv(f'{data_path}train_120000_ver1_kiwi_str.csv', index=False, encoding='utf-8-sig' )